# BNPL Data Exploration

**Objective**: Comprehensive exploratory data analysis of BNPL staging data

**Data Source**: `flit-data-platform.flit_staging.stg_bnpl_raw_transactions`
- **Records**: ~1.9M transactions
- **Fields**: 42 columns
- **Time Range**: TBD

**Research Questions**:
1. What is the distribution of transaction amounts?
2. How do customer demographics correlate with risk?
3. What temporal patterns exist in BNPL usage?
4. What are the key predictive features for default risk?

In [ ]:
# Environment setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# BigQuery integration
from google.cloud import bigquery
from flit_ml.config import config

# Configuration
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("Environment setup complete!")

## 1. Data Connection and Schema Analysis

In [ ]:
# Connect to BigQuery
client = config.get_client()

# Test connection with basic query
test_query = """
SELECT COUNT(*) as total_records
FROM `flit-data-platform.flit_staging.stg_bnpl_raw_transactions`
"""

result = client.query(test_query).result()
for row in result:
    print(f"✅ Connected to BigQuery: {row.total_records:,} total records")

In [ ]:
# Get table schema information
schema_query = """
SELECT
    column_name,
    data_type,
    is_nullable,
    description
FROM `flit-data-platform.flit_staging.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'stg_bnpl_raw_transactions'
ORDER BY ordinal_position
"""

schema_df = client.query(schema_query).to_dataframe()
print(f"📊 Schema: {len(schema_df)} columns")
schema_df.head(20)

## 2. Sample Data Inspection

In [ ]:
# Get sample records for initial inspection
sample_query = """
SELECT *
FROM `flit-data-platform.flit_staging.stg_bnpl_raw_transactions`
ORDER BY RAND()
LIMIT 1000
"""

# Load sample data
print("📥 Loading sample data...")
sample_df = client.query(sample_query).to_dataframe()

print(f"Sample data shape: {sample_df.shape}")
print(f"Memory usage: {sample_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

sample_df.head()

In [ ]:
# Data types and basic info
print("📋 Data Types and Info:")
sample_df.info()

In [ ]:
# Statistical summary for numeric columns
print("📈 Statistical Summary:")
sample_df.describe()

## 3. Data Quality Assessment

In [ ]:
# Missing values analysis
missing_stats = {
    'column': sample_df.columns,
    'missing_count': sample_df.isnull().sum(),
    'missing_percent': (sample_df.isnull().sum() / len(sample_df) * 100).round(2),
    'dtype': sample_df.dtypes
}

missing_df = pd.DataFrame(missing_stats)
missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_percent', ascending=False)

print("❓ Missing Values Analysis:")
if len(missing_df) > 0:
    print(missing_df)
else:
    print("✅ No missing values found in sample data")

In [ ]:
# Unique values count for potential categorical variables
print("🏷️  Categorical Variables Analysis:")

categorical_analysis = []
for col in sample_df.columns:
    unique_count = sample_df[col].nunique()
    unique_ratio = unique_count / len(sample_df)
    
    # Identify potential categorical variables
    if unique_ratio < 0.1 and unique_count < 50:
        categorical_analysis.append({
            'column': col,
            'unique_count': unique_count,
            'unique_ratio': round(unique_ratio, 3),
            'sample_values': list(sample_df[col].value_counts().head(5).index)
        })

if categorical_analysis:
    cat_df = pd.DataFrame(categorical_analysis)
    for _, row in cat_df.iterrows():
        print(f"{row['column']}: {row['unique_count']} unique values ({row['unique_ratio']*100:.1f}%)")
        print(f"  Sample values: {row['sample_values']}")
        print()
else:
    print("No obvious categorical variables detected in sample")

## 4. Key Business Metrics Overview

Understanding the business context of BNPL transactions

In [ ]:
# Basic business metrics from full dataset
business_metrics_query = """
SELECT
    COUNT(*) as total_transactions,
    COUNT(DISTINCT customer_id) as unique_customers,
    AVG(transaction_amount) as avg_transaction_amount,
    STDDEV(transaction_amount) as std_transaction_amount,
    MIN(transaction_amount) as min_transaction_amount,
    MAX(transaction_amount) as max_transaction_amount,
    PERCENTILE_CONT(transaction_amount, 0.25) OVER() as p25_transaction_amount,
    PERCENTILE_CONT(transaction_amount, 0.5) OVER() as median_transaction_amount,
    PERCENTILE_CONT(transaction_amount, 0.75) OVER() as p75_transaction_amount,
    MIN(transaction_date) as earliest_transaction,
    MAX(transaction_date) as latest_transaction
FROM `flit-data-platform.flit_staging.stg_bnpl_raw_transactions`
LIMIT 1
"""

print("📊 Business Metrics Overview:")
business_metrics = client.query(business_metrics_query).to_dataframe()
business_metrics.T

## 5. Next Steps for Deep Analysis

Based on the initial exploration, identify key areas for detailed analysis:

1. **Transaction Amount Distribution**: Understand spending patterns
2. **Customer Segmentation**: Identify different customer behaviors
3. **Temporal Patterns**: Seasonal and time-based trends
4. **Risk Indicators**: Identify features correlated with defaults
5. **Feature Engineering**: Create BNPL-specific features

In [ ]:
# Save key insights for next notebook
insights = {
    'total_records': int(business_metrics['total_transactions'].iloc[0]),
    'unique_customers': int(business_metrics['unique_customers'].iloc[0]),
    'avg_transaction': float(business_metrics['avg_transaction_amount'].iloc[0]),
    'date_range': {
        'start': str(business_metrics['earliest_transaction'].iloc[0]),
        'end': str(business_metrics['latest_transaction'].iloc[0])
    },
    'schema_columns': len(schema_df),
    'sample_size': len(sample_df)
}

print("💾 Key Insights Summary:")
for key, value in insights.items():
    print(f"{key}: {value}")

# TODO: Save insights to file for next notebooks
# import json
# with open('../reports/data_exploration_insights.json', 'w') as f:
#     json.dump(insights, f, indent=2)